In [1]:
import numpy as np
import pandas as pd

In [52]:
shops_df = pd.read_csv("./datasets/shops.csv")
items_df = pd.read_csv("./datasets/items.csv")
item_categories_df = pd.read_csv("./datasets/item_categories.csv")
sale_train_df = pd.read_csv("./datasets/sales_train.csv")
test_df = pd.read_csv("./datasets/test.csv")
sample_submission = pd.read_csv("./datasets/sample_submission.csv")

In [53]:
shops_df.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [54]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [55]:
item_categories_df.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [70]:
sale_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [71]:
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [72]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [73]:
# result = sale_train_df[[ 'date_block_num','shop_id','item_id','item_price','item_cnt_day']].groupby(['date_block_num', "shop_id", 'item_id', 'item_price',], 
#                                                                      as_index=False).count()
# result

In [74]:
price_df = sale_train_df[['date_block_num', "shop_id", "item_id",'item_price']].groupby(["date_block_num","shop_id", "item_id"], 
                                                                    as_index=False).mean()

In [75]:
item_cnt_day_df = sale_train_df[['date_block_num', "shop_id", "item_id",'item_cnt_day']].groupby(["date_block_num","shop_id", "item_id"], 
                                                                    as_index=False).sum()

In [76]:
price_df["item_cnt_day"] = item_cnt_day_df['item_cnt_day']
price_df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,0,32,221.0,6.0
1,0,0,33,347.0,3.0
2,0,0,35,247.0,1.0
3,0,0,43,221.0,1.0
4,0,0,51,128.5,2.0


In [77]:
sale_train_df.loc[(sale_train_df["shop_id"] == 5)
                 & (sale_train_df["item_id"] == 5232)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2810654,28.08.2015,31,5,5232,599.0,1.0


In [80]:
selected_features = ['item_id','shop_id' ]
X_train = sale_train_df[selected_features]
Y_train = sale_train_df["item_cnt_day"]

In [81]:
X_test = test_df[selected_features]

In [82]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

In [83]:
# 进行特征向量化
from sklearn.feature_extraction import DictVectorizer
dict_vec = DictVectorizer(sparse=False)

X_train = dict_vec.fit_transform(X_train.to_dict(orient='record'))
X_test = dict_vec.transform(X_test.to_dict(orient='record'))

In [88]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [103]:
from sklearn.ensemble import GradientBoostingRegressor
rfr = GradientBoostingRegressor()
rfr.fit(X_train, Y_train)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [104]:
rfr_y_predict = rfr.predict(X_test)

In [105]:
rfr_submission = pd.DataFrame({'ID':test_df['ID'], 'item_cnt_month': rfr_y_predict})


,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,1.189295
std,61834.358168,0.308923
min,0.000000,0.683101
25%,53549.750000,1.090492
50%,107099.500000,1.131339
75%,160649.250000,1.179677
max,214199.000000,4.581892


In [107]:
rfr_submission['item_cnt_month'] = np.floor(rfr_submission['item_cnt_month'])

In [108]:
rfr_submission.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,1.030616
std,61834.358168,0.279867
min,0.000000,0.000000
25%,53549.750000,1.000000
50%,107099.500000,1.000000
75%,160649.250000,1.000000
max,214199.000000,4.000000


In [109]:
rfr_submission.to_csv('./sub.csv', index=False)